In [3]:
# read data from crest format
import pandas as pd
df = pd.read_excel(r'simeval5.xlsx')
json_test = []
json_train = []

In [4]:
# convert crest into cnn format
def find_index_in_sentence(sentence, searched_word):
    for i, word in enumerate(sentence):
        if word == searched_word:
            return i
    return -1

def extract_start_end_position(sentence, span):
    span = span.split()
    if len(span) == 0:
        return -1, -1
    else:
        start = find_index_in_sentence(sentence, span[0])
    if len(span) > 1:
        end = find_index_in_sentence(sentence, span[-1])
    else:
        end = start
    return start, end
for index, row in df.iterrows():
    line = {}
    line['id'] = str(row['global_id'])
    line['sentence'] = row['context'].split(" ")
    for i in range(len(line['sentence'])):
        line['sentence'][i] = ''.join([i for i in line['sentence'][i] if i.isalpha()])
    line['sentence'].append(".")
    span1, span2 = row['span1'][2:-2], row['span2'][2:-2]
    line['head'] = span1
    line['tail'] = span2
    line['subj_start'], line['subj_end'] = extract_start_end_position(line['sentence'], span1)
    line['obj_start'], line['obj_end'] = extract_start_end_position(line['sentence'], span2)
    if row['label'] == 1:
        if row['direction'] == 0:
            line['relation'] = "Cause-Effect(e1,e2)"
        else:
            line['relation'] = "Cause-Effect(e2,e1)"
    else:
        line['relation'] = "Other"
    if row['split'] == 0:
        json_train.append(line)
    else:
        json_test.append(line)

In [5]:
# save train.json; put it into CNN/data/
import json
with open("train.json", "w") as json_file:
    for line in json_train:
        json_file.write(json.dumps(line))
        json_file.write("\n")

In [6]:
# save test.json; put it into CNN/data/
with open("test.json", "w") as json_file:
    for line in json_test:
        json_file.write(json.dumps(line))
        json_file.write("\n")